Nama: Khresna Pandu Izzaturrahman

NIM: 18_430266_PA_18779



In [1]:
#import library yang dibutuhkan
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Import dataset dan tokenization pada kolom title**

Import dataset menggunakan library pandas dan diubah menjadi dataframe dengan nama 'okezone'.

Untuk tokenization dibuat fungsi 'tokenization' dengan parameter kolom pada dataframe, dan dataframe. Didalam fungsi 'tokenization' dilakukan fungsi lambda untuk men-tokenize setiap kolom (yang dimasukan pada parameter) menggunakan library nltk dan disimpan pada kolom baru 'tokenized_title'.

In [2]:
#Import dataset dan dimasukan ke dataframe okezone
okezone = pd.read_csv('/content/drive/My Drive/KS Sistem Cerdas/annotated_okezone.csv')
print(okezone.loc[1, :])
#Training phase

#Fungsi untuk tokenization dari suatu kolom dataframe
def tokenization(column, df):
  #Membuat kolom baru untuk menyimpan tokenization
  df['tokenized_title'] = df.apply(lambda row: nltk.word_tokenize(row[column]), axis=1)
  return df

#Membuat token
okezone = tokenization('title', okezone)

#Mencoba hasil setelah tokenization
print('Hasil tokenization pada baris pertama')
print(okezone.loc[0, 'tokenized_title'])

for i in range(len(okezone.loc[0,'tokenized_title'])):
  print(okezone.loc[0, 'tokenized_title'][i])

title          Balita di Bogor Tewas dengan Luka Lebam, Ibu T...
label                                              non-clickbait
label_score                                                    0
Name: 1, dtype: object
Hasil tokenization pada baris pertama
['Ini', 'Penyebab', 'Jamaah', 'Tertipu', 'Penggunaan', 'Visa', 'Non-Haji']
Ini
Penyebab
Jamaah
Tertipu
Penggunaan
Visa
Non-Haji


**Membuat Vocabulary**

Alur untuk membuat vocabulary adalah 


1.   Menggabungkan semua token pada setiap baris dataframe ke satu list vocabulary menggunakan fungsi 'build_vocab'
2.   Melakukan cleaning text pada list vocabulary menggunakan fungsi 'cleaning_text'. Cleaning text yang dilakukan adalah menghapus tanda baca, meng-lowercase semua kata, dan menghapus stop words.
3.   Mengambil 1000 kata paling umum  dalam vocabulary dan mengabaikan diluar 1000 kata paling umum.


In [3]:
# Fungsi untuk menggabungkan semua token pada setiap baris dataframe ke satu list vocabulary
def build_vocab(column, df):
  vocab_wannabe = []
  for i in range(len(df[column])):
    vocab_wannabe += df.loc[i, column]
  return vocab_wannabe

#Membuat vocabulary
vocabulary = build_vocab('tokenized_title', okezone)
print('Vocabulary semula\n', vocabulary[:50])
print()

from nltk.corpus import stopwords
stop_words = set(stopwords.words('indonesian'))
#Fungsi cleaning text
def cleaning_text(vocab):

  #Menghapus tanda baca pada vocabulary
  vocab = [word for word in vocab if word.isalpha()]
  #Meng-lowercase semua kata pada vocabulary
  vocab = [word.lower() for word in vocab]
  #Menghapus stop words
  vocab = [word for word in vocab if not word in stop_words]
  return vocab

#Fungsi untuk mengambil 1000 kata paling umum dan unik
def unique(vocab):
  frequent = nltk.FreqDist(vocab)
  frequent = frequent.most_common(1000)
  frequent = [word for word, word_count in frequent]
  return frequent

#Fungsi ini pertama-tama mengambil 1000 kata unik yang paling umum
#kemudian menghapus kata-kata pada vocabulary yang tidak termasuk 
#1000 kata paling umum
def not_unique(vocab):
  vocab_baru = []
  #Mengambil 1000 kata unik yang paling umum
  #kemudian disimpan pada variabel frequent 
  frequent = nltk.FreqDist(vocab)
  frequent = frequent.most_common(1000)

  #variabel vocab adalah vocabulary utuh
  #variabel frequent adalah 1000 kata paling umum DAN unik

  #Menghapus kata-kata yang tidak termasuk 1000
  #kata paling umum dari vocabulary
  for i in range(len(vocab)):
    for j in range(len(frequent)):
      if vocab[i] == frequent[j][0]:
        vocab_baru.append(vocab[i])
  
  #variabel vocab_baru menyimpan 1000 kata paling umum
  #dan tidak unik
  return vocab_baru 

#Membuat fungsi yang mengubah kolom pada dataframe menjadi satu list vocabulary
#yang sudah clean dan mengambil 1000 kata paling umum
def column_to_vocab(column, df):
  df = tokenization(column, df)
  VocabularY = build_vocab('tokenized_title', df)
  Vocabulary = cleaning_text(VocabularY)
  Vocabulary = not_unique(Vocabulary)
  return Vocabulary

#Hasil setelah dibuat vocabulary
vocabulary = cleaning_text(vocabulary)
print('Setelah dilakukan cleaning')
print(len(vocabulary))
print(vocabulary[:50])
print()

print('vocabulary dengan 1000 kata paling umum')
vocabulary = unique(vocabulary)
print(len(vocabulary))
print(vocabulary[:50]) 

Vocabulary semula
 ['Ini', 'Penyebab', 'Jamaah', 'Tertipu', 'Penggunaan', 'Visa', 'Non-Haji', 'Balita', 'di', 'Bogor', 'Tewas', 'dengan', 'Luka', 'Lebam', ',', 'Ibu', 'Tiri', 'Ditetapkan', 'Tersangka', 'Demi', 'Keadilan', ',', 'Pria', 'Ini', 'Habiskan', 'Rp526', 'Juta', 'Lawan', 'Denda', 'Tilang', 'Rp1', 'Juta', 'Claudia', 'Emanuela', 'Santoso', 'Harumkan', 'Indonesia', 'di', 'The', 'Voice', 'of', 'Germany', 'Kalah', 'Saing', 'di', 'Monza', ',', 'Bottas', 'Akui', 'Ketangguhan']

Setelah dilakukan cleaning
10118
['penyebab', 'jamaah', 'tertipu', 'penggunaan', 'visa', 'balita', 'bogor', 'tewas', 'luka', 'lebam', 'tiri', 'ditetapkan', 'tersangka', 'keadilan', 'pria', 'habiskan', 'juta', 'lawan', 'denda', 'tilang', 'juta', 'claudia', 'emanuela', 'santoso', 'harumkan', 'indonesia', 'the', 'voice', 'of', 'germany', 'kalah', 'saing', 'monza', 'bottas', 'akui', 'ketangguhan', 'leclerc', 'afc', 'solidarity', 'cup', 'jalur', 'alternatif', 'timnas', 'indonesia', 'piala', 'asia', 'bertitel', 'edis

**Menghitung dua parameter**

Alur dalam menghitung dua parameter adalah


1.   Membuat dataframe baru berdasarkan nilai pada suatu kolom menggunakan fungsi 'new_dataframe'.
2.   Menghitung prior probability masing-masing kelas dengan cara membagi panjang dataframe given class j dengan dataframe yang utuh
3.   Menghitung conditional probability setiap kata given class j dengan fungsi 'word_conditional_prob'. Hasilnya disimpan dalam bentuk dictionary dengan keys adalah masing-masing kata dan values berupa conditional probabilitynya.





In [4]:
#Menghitung prior probability

#Fungsi untuk membuat dataframe baru berdasarkan nilai pada suatu kolom
def new_dataframe(df, column, value):
  new_df = df[df[column]==value].reset_index(drop=True)
  return new_df

#Fungsi untuk menghitung prior probability given class
def prior_prob(df, df_given_class):
  prior_probability = len(df_given_class)/len(df)
  return prior_probability

#Membuat dataframe baru yang berisi title clickbait
df_clickbait = new_dataframe(okezone, 'label_score', 1)

#Membuat dataframe baru yang berisi title non-clickbait
df_nonclickbait = new_dataframe(okezone, 'label_score', 0)

#Menghitung prior probability suatu title adalah clickbait
prior_prob_clickbait = prior_prob(okezone, df_clickbait)
print('Prior probability clickbait = ', prior_prob_clickbait)

#Menghitung prior probability suatu title bukan clickbait
prior_prob_nonclickbait = prior_prob(okezone, df_nonclickbait)
print('Prior probability nonclickbait = ', prior_prob_nonclickbait)

#Conditional probability untuk setiap kata pada dokumen (title)

#Fungsi untuk menghitung nilai denominator pada suatu word conditional 
def denominator(vocab):
  #Membuat dictionary dengan keys adalah unique word dan values berupa frekuensi
  #kata tersebut
  fdist = nltk.FreqDist(vocab)
  word_freq = dict((word, freq) for word, freq in fdist.items() if not word.isdigit())

  #Menghitung total kata pada suatu vocabulary
  jumlah_kata = 0
  for i in word_freq:
    jumlah_kata += word_freq[i]

  return (jumlah_kata + len(vocab))

#Fungsi menghitung probabilitas setiap kata pada suatu vocab
def word_conditional_prob(vocab):

  #Membuat dictionary dengan keys adalah unique word dan values berupa frekuensi
  #kata tersebut
  fdist = nltk.FreqDist(vocab)
  word_freq = dict((word, freq) for word, freq in fdist.items() if not word.isdigit())

  #Menghitung total kata pada suatu vocabulary
  jumlah_kata = 0
  for i in word_freq:
    jumlah_kata += word_freq[i]

  #Menghitung conditional probability setiap kata, dengan keys adalah unique word
  #dan values berupa probabilitasnya
  for i in word_freq:
    #ditambah 1 untuk add one smoothing
    word_freq[i] = (word_freq[i]+1) / (jumlah_kata + len(vocab))

  return word_freq


#Membuat vocabulary dengan 1000 kata paling umum, 1 unik 1 lagi tidak unik
nonunique_clickbait_vocab = column_to_vocab('title', df_clickbait)
unique_clickbait_vocab = unique(nonunique_clickbait_vocab)
 
#Menghitung probabilitas kemunculan kata given clickbait
clickbait_conditional_prob = word_conditional_prob(nonunique_clickbait_vocab)
print('Clickbait conditional probability ', clickbait_conditional_prob)
print()

#Membuat vocabulary dengan 1000 kata paling umum, 1 unik 1 lagi tidak unik
nonunique_nonclickbait_vocab = column_to_vocab('title', df_nonclickbait)
unique_nonclickbait_vocab = unique(nonunique_nonclickbait_vocab)

#Menghitung probabilitas kemunculan kata given non-clickbait
nonclickbait_conditional_prob = word_conditional_prob(nonunique_nonclickbait_vocab)
print('nonclickbait conditional probability', nonclickbait_conditional_prob)
print()

print(len(nonunique_clickbait_vocab))
print(nonunique_clickbait_vocab)
print(len(unique_clickbait_vocab))
print(unique_clickbait_vocab)

Prior probability clickbait =  0.506
Prior probability nonclickbait =  0.494
Clickbait conditional probability  {'penyebab': 0.0011689070718877848, 'jamaah': 0.0007305669199298656, 'tertipu': 0.00043834015195791935, 'penggunaan': 0.00043834015195791935, 'visa': 0.0002922267679719462, 'keadilan': 0.0002922267679719462, 'pria': 0.0029222676797194622, 'habiskan': 0.0005844535359438924, 'juta': 0.001315020455873758, 'lawan': 0.0007305669199298656, 'denda': 0.0002922267679719462, 'tilang': 0.00043834015195791935, 'titi': 0.0002922267679719462, 'dj': 0.00043834015195791935, 'borong': 0.0002922267679719462, 'peserta': 0.00043834015195791935, 'blind': 0.0002922267679719462, 'audition': 0.0002922267679719462, 'the': 0.0005844535359438924, 'voice': 0.00043834015195791935, 'indonesia': 0.005552308591466978, 'episode': 0.0002922267679719462, 'sukses': 0.0008766803039158387, 'riri': 0.0002922267679719462, 'muha': 0.0002922267679719462, 'hadirkan': 0.00043834015195791935, 'lagu': 0.00102279368790181

**Memprediksi kelas pada test set (CMAP)**

Alur untuk memprediksi kelas setiap title adalah


1.   Menyimpan tes set ke dataframe fimela
2.   Melakukan tokenization pada kolom 'title' dan disimpan dalam bentuk kolom baru 'tokenized_title'.
3.   Melakukan cleaning text pada kolom 'tokenized_title'.
4.   Menghitung prior probability class j yang didapat pada traning set sebelumnya.
5.   Menghitung probabilitas setiap title untuk clikcbait dan disimpan pada kolom baru 'clickbait_probability'. Kemudian dilakukan hal  yang sama untuk menghitung probabilitas nonclickbait dan disimpan pada title baru 'nonclickbait_probability'
6.   Jika 'clickbait_probability' lebih besar dari 'nonclickbait_probability' maka prediksi untuk title tersebut adalah clickbait dan disimpan pada kolom baru 'prediksi' dengan nilai 1. Jika tidak, maka kolom 'prediksi' bernilai 0
7.   Membandingkan nilai kolom 'prediksi' dan 'label_score' untuk menghitung akurasi.










In [6]:
#Membuat dataframe fimela
fimela = pd.read_csv('/content/drive/My Drive/KS Sistem Cerdas/annotated_fimela.csv')

#Tokenization pada pada kolom title dan disimpan pada kolom baru
#'tokenized_title'
fimela = tokenization('title', fimela)

#Melakukan cleaning text pada kolom 'tokenized_title'
for i in range(len(fimela['tokenized_title'])):
  fimela.at[i, 'tokenized_title'] = cleaning_text(fimela.loc[i, 'tokenized_title'])

#Menghitung probabilitas suatu title adalah clickbait dan disimpan pada
#kolom baru clickbait_prob

#Clickbait_prob adalah probabilitas suatu title adalah clickbait
clickbait_prob = prior_prob_clickbait

#iterasi pada setiap baris
for i in range(len(fimela)):
  #prior probability
  clickbait_prob = prior_prob_clickbait

  #iterasi pada setiap kata pada title
  for j in range(len(fimela.loc[i, 'tokenized_title'])):
    #iterasi pada setiap kata pada vocabulary (probabilitas masing2 kata)
    for k, l in clickbait_conditional_prob.items():
   
      #jika kata j pada title terdapat dalam vocabulary, dihitung
      #conditional probabilitynya
      if (fimela.loc[i, 'tokenized_title'][j] == k):
        clickbait_prob = clickbait_prob*l
   
    #jika kata j tidak ditemukan, dianggap satu untuk add one smoothing
    if clickbait_prob == prior_prob_clickbait:
      clickbait_prob = clickbait_prob*(1/denominator(nonunique_clickbait_vocab))
    
  fimela.at[i, 'clickbait_probability'] = clickbait_prob
  print('selesai baris ke-'+str(i)+' dengan probability clickbait = ' +str(clickbait_prob))

#Menghitung probabilitas setiap title given non clickbait
nonclickbait_prob = prior_prob_nonclickbait

for i in range(len(fimela)):
  #prior probability
  nonclickbait_prob = prior_prob_nonclickbait
  #iterasi pada setiap kata pada title
  for j in range(len(fimela.loc[i, 'tokenized_title'])):
    #iterasi pada setiap kata pada vocabulary (probabilitas masing2 kata)
    for k, l in nonclickbait_conditional_prob.items():
   
      #jika kata j pada title terdapat dalam vocabulary, dihitung
      #conditional probabilitynya
      if (fimela.loc[i, 'tokenized_title'][j] == k):
        nonclickbait_prob = nonclickbait_prob*l
   
    #jika kata j tidak ditemukan, dianggap satu untuk add one smoothing
    if nonclickbait_prob == prior_prob_nonclickbait:
      nonclickbait_prob = nonclickbait_prob*(1/denominator(nonunique_nonclickbait_vocab))
    
  fimela.at[i, 'nonclickbait_probability'] = nonclickbait_prob
  print('selesai baris ke-'+str(i)+' dengan probability nonclickbait = ' +str(nonclickbait_prob))

#Iterasi untuk menentukan apakah suatu title clickbait atau bukan
for i in range(len(fimela)):
  #jika probabilitas clickbait lebih besar dari nonclickbait
  #maka title tersebut adalah clickbait
  if fimela.loc[i, 'clickbait_probability'] >= fimela.loc[i, 'nonclickbait_probability']:
    fimela.at[i, 'prediksi'] = int(1)
  #jika tidak, maka nonclickbait
  else:
    fimela.at[i, 'prediksi'] = int(0)

#Menentukan akurasi prediksi
jumlah_benar = 0
for i in range(len(fimela)):
  #jika prediksi sama dengan label_score jumlah benar = 1
  if fimela.loc[i, 'prediksi'] == fimela.loc[i, 'label_score']:
    jumlah_benar += 1
print('akurasi = ', jumlah_benar/len(fimela))

selesai baris ke-0 dengan probability clickbait = 1.2963186258954248e-07
selesai baris ke-1 dengan probability clickbait = 3.7881900230725444e-11
selesai baris ke-2 dengan probability clickbait = 1.7678220107671876e-10
selesai baris ke-3 dengan probability clickbait = 4.537115190633987e-07
selesai baris ke-4 dengan probability clickbait = 0.0001478667445938048
selesai baris ke-5 dengan probability clickbait = 4.321062086318082e-08
selesai baris ke-6 dengan probability clickbait = 8.839110053835938e-11
selesai baris ke-7 dengan probability clickbait = 0.00022180011689070718
selesai baris ke-8 dengan probability clickbait = 4.321062086318082e-08
selesai baris ke-9 dengan probability clickbait = 4.261713775956613e-11
selesai baris ke-10 dengan probability clickbait = 8.642124172636165e-08
selesai baris ke-11 dengan probability clickbait = 2.841142517304409e-10
selesai baris ke-12 dengan probability clickbait = 2.5254600153816963e-10
selesai baris ke-13 dengan probability clickbait = 7.393

**Micro averaging and macroaveraging**

Alur untuk menghitung micro averaging dan macroaveragin adalah


1.   Menghitung TP, FP, TN, FN untuk kelas clickbait
2.   Menghitung TP, FP, TN, FN untuk kelas nonclickbait
3.   Menghitung nilai macroaveraging
4.   Menghitung nilai microaveraging





In [ ]:
#Menghitung TP, FP, TN, FN untuk kelas clickbait
TP_clickbait = 0
FP_clickbait = 0
TN_clickbait = 0
FN_clickbait = 0

for i in range(len(fimela)):
  #Menghitung nilai TP
  if (fimela.loc[i, 'label_score']==1 and fimela.loc[i, 'prediksi']==1):
    TP_clickbait += 1
  #Menghitung nilai FP
  elif (fimela.loc[i, 'label_score']==0 and fimela.loc[i, 'prediksi']==1):
    FP_clickbait += 1
  #Menghitung nilai TN
  elif (fimela.loc[i, 'label_score']==0 and fimela.loc[i, 'prediksi']==0):
    TN_clickbait += 1
  #Menghitung nilai FN
  elif (fimela.loc[i, 'label_score']==1 and fimela.loc[i, 'prediksi']==0):
    FN_clickbait +=1

#Menghitung TP, FP, TN, FN untuk kelas nonclickbait
TP_nonclickbait = 0
FP_nonclickbait = 0
TN_nonclickbait = 0
FN_nonclickbait = 0

for i in range(len(fimela)):
  #Menghitung nilai TP
  if (fimela.loc[i, 'label_score']==0 and fimela.loc[i, 'prediksi']==0):
    TP_nonclickbait += 1
  #Menghitung nilai FP
  elif (fimela.loc[i, 'label_score']==1 and fimela.loc[i, 'prediksi']==0):
    FP_nonclickbait += 1
  #Menghitung nilai TN
  elif (fimela.loc[i, 'label_score']==1 and fimela.loc[i, 'prediksi']==1):
    TN_nonclickbait += 1
  #Menghitung nilai FN
  elif (fimela.loc[i, 'label_score']==0 and fimela.loc[i, 'prediksi']==1):
    FN_nonclickbait += 1

#Menghitung nilai macroaveraging
clickbait_precision = TP_clickbait/(TP_clickbait + FP_clickbait)
nonclickbait_precision = TP_nonclickbait/(TP_nonclickbait + FP_nonclickbait)
macroaveraged_precision = (clickbait_precision + nonclickbait_precision)/2
print('macroaveraged precision = ', macroaveraged_precision)

#Menghitung nilai microaveraging
microaveraged_precision = (TP_clickbait + TP_nonclickbait)/(TP_clickbait + TP_nonclickbait + FP_clickbait + FP_nonclickbait)
print('microaveraged precision = ', microaveraged_precision)

macroaveraged precision =  0.5416268935463789
microaveraged precision =  0.5328571428571428


**Analisis**

Setelah dilakukan traning dan diuji coba pada test set, akurasi yang didapatkan masih rendah yaitu 0.5328. Hal yang mungkin menyebabkan rendahnya akurasi adalah outlet media yang berbeda antara training set dan test set, perbedaan outlet tersebut mungkin menyebabkan gaya bahasa yang berbeda sehingga kata-kata yang probabilitas clickbaitnya tinggi pada training set belum tentu berlaku hal yang sama pada test set. Dataset yang tidak besar mungkin juga mempengaruhi rendahnya akurasi.

Hal lain yang mungkin menyebabkan rendahnya akurasi adalah kurang atau terdapat kesalahan pada preprocessing text. Perlu dilakukan trial error dan mencoba teknik lain dalam preprocessing untuk melihat efeknya pada akurasi.

Penyebab lain yang mungkin adalah kesalahan pada algoritma yang tidak disadari oleh saya.